(COMPC)=

# 5.3 Compilación a C

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota el y la lectora:
:class: tip

* 

```

Los lenguajes de programación y sus implementaciones pueden tener características como las siguientes:

* Realizar un *parsing* de las instrucciones y ejecutar las acciones de forma casi inmediata (intérprete). Como ejemplo está [Beginners' All-purpose Symbolic Instruction Code: BASIC](https://en.wikipedia.org/wiki/BASIC)

* Realizar un *parsing* de las instrucciones, traducir a una [representación intermedia](https://en.wikipedia.org/wiki/Intermediate_representation) y ejecutar las acciones. Como ejemplo se encuentra *Python* en su implementación *CPython*. La interpretación de código intermedia es un [bytecode](https://en.wikipedia.org/wiki/Bytecode).

* Compilar [ahead of time](https://en.wikipedia.org/wiki/Ahead-of-time_compilation) las instrucciones antes de su ejecución. Como ejemplo se encuentran *C, C++* y *Fortran*.

* Realizar un *parsing* de las instrucciones y compilar las instrucciones en una forma [just in time compilation](https://en.wikipedia.org/wiki/Just-in-time_compilation) y *at* [runtime](https://en.wikipedia.org/wiki/Runtime_(program_lifecycle_phase)). Como ejemplos se encuentran *Julia* y *Python* en su implementación con [PyPy](https://en.wikipedia.org/wiki/PyPy).

La ejecución de instrucciones en los lenguajes de programación será más rápida dependiendo del lenguaje, la implementación que se utilice del mismo y *features* que tenga el lenguaje. 

## Python

In [1]:
%%file Rcf_python.py
import math
import time
def Rcf(f,a,b,n): #Rcf: rectángulo compuesto para f
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat = (b-a)/n
    sum_res = 0
    for i in range(n):
        x = a + (i + 1/2)*h_hat
        sum_res += f(x)
    return h_hat*sum_res

if __name__ == "__main__":   
    n = 10**7
    f = lambda x: math.exp(-x**2)
    a = 0
    b = 1
    start_time = time.time()
    res = Rcf(f,a,b,n)
    end_time = time.time()
    secs = end_time-start_time
    print("Rcf tomó", secs, "segundos" )

Writing Rcf_python.py


In [2]:
%%bash
python3 Rcf_python.py

Rcf tomó 3.4265081882476807 segundos


## C

Para la medición de tiempos se utilizaron las ligas: [liga](https://stackoverflow.com/questions/16764276/measuring-time-in-millisecond-precision) y [liga2](https://www.techiedelight.com/find-execution-time-c-program/).

In [3]:
%%file Rcf_c.c
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<time.h>
#include <sys/time.h>

void Rcf(double ext_izq, double ext_der, int n,\
    double *suma_global_p);
double f(double nodo);
int main(int argc, char *argv[]){
    double suma_global = 0.0;
    double a = 0.0, b = 1.0;
    int n = 1e7;
    struct timeval start;
    struct timeval end;
    long seconds;
    long long mili;
    
    gettimeofday(&start, NULL);
    Rcf(a,b,n,&suma_global);
    gettimeofday(&end, NULL);
    seconds = (end.tv_sec - start.tv_sec);
    mili = 1000*(seconds) + (end.tv_usec - start.tv_usec)/1000;    
    printf("Tiempo de ejecución: %lld milisegundos", mili);
    
    return 0;
}
void Rcf(double a, double b, int n, double *sum){
    double h_hat = (b-a)/n;
    double x = 0.0;
    int i = 0;
    *sum = 0.0;
    for(i = 0; i <= n-1; i++){
        x = a + (i + 1/2.0)*h_hat;
        *sum += f(x);
    }
    *sum = h_hat*(*sum);
}
double f(double nodo){
    double valor_f;
    valor_f = exp(-pow(nodo,2));
    return valor_f;
}


Writing Rcf_c.c


In [4]:
%%bash
gcc -Wall Rcf_c.c -o Rcf_c.out -lm

In [5]:
%%bash
./Rcf_c.out

Tiempo de ejecución: 482 milisegundos

## R

In [6]:
%%file Rcf_R.R
library(tictoc)
Rcf<-function(f,a,b,n){
    '
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (float): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    '
    
    h_hat <- (b-a)/n
    sum_res <- 0
    for(i in 0:(n-1)){
        x <- a + (i + 1/2)*h_hat
        sum_res <- sum_res + f(x)
    }
    approx <- h_hat*sum_res
}
n <- 10**7
f <- function(x)exp(-x^2)
a <- 0
b <- 1
tic("Rcf tomó")
Rcf(f,a,b,n)
toc()

Writing Rcf_R.R


In [7]:
%%bash
Rscript Rcf_R.R

Rcf tomó: 5.568 sec elapsed


## Julia

In [8]:
%%file Rcf_julia.jl
"""
Compute numerical approximation using rectangle or mid-point
method in an interval.

# Arguments

- `f::Float`: function expression of integrand.
- `a::Integer`: left point of interval.
- `b::Integer`: right point of interval.
- `n::Integer`: number of subintervals.
"""
function Rcf(f, a, b, n)
    h_hat = (b-a)/n
    sum_res = 0
    for i in 0:n-1
        x = a + (i + 1/2)*h_hat
        sum_res += f(x)
    end    
    return h_hat*sum_res
end
function main()
    a = 0
    b = 1
    n =10^7
    f(x) = exp(-x^2)
    @time Rcf(f, a, b, n)
    @time Rcf(f, a, b, n)
end

main()

Writing Rcf_julia.jl


In [9]:
%%bash
/usr/local/julia-1.6.0/bin/julia Rcf_julia.jl

  0.228430 seconds
  0.228310 seconds


In [10]:
%%file Rcf_julia_typed_values.jl
"""
Compute numerical approximation using rectangle or mid-point
method in an interval.

# Arguments

- `f::Float`: function expression of integrand.
- `a::Integer`: left point of interval.
- `b::Integer`: right point of interval.
- `n::Integer`: number of subintervals.
"""
function Rcf(f, a, b, n)
    h_hat = (b-a)/n
    sum_res = 0.0
    for i in 0:n-1
        x = a + (i + 1/2)*h_hat
        sum_res += f(x)
    end    
    return h_hat*sum_res
end
function main()
    a = 0
    b = 1
    n =10^7
    f(x) = exp(-x^2)
    @time Rcf(f, a, b, n)
    @time Rcf(f, a, b, n)    
end

main()

Writing Rcf_julia_typed_values.jl


In [11]:
%%bash
/usr/local/julia-1.6.0/bin/julia Rcf_julia_typed_values.jl

  0.092623 seconds
  0.092402 seconds


## [cython](https://cython.org/)

## Referencias de interés

* [introduction to cython](http://okigiveup.net/an-introduction-to-cython/).

* [Basic cython tutorial](https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

* [Source files and compilation](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html)

* [Compiling with a jupyter notebook](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiling-with-a-jupyter-notebook)

* [Dirk Eddelbuettel: rcpp ](http://dirk.eddelbuettel.com/code/rcpp.html).

* [Rcpp for everyone](https://teuder.github.io/rcpp4everyone_en/).

* [Introduction to rcpp:From Simple Examples to Machine Learning](http://dirk.eddelbuettel.com/papers/rcpp_rfinance_may2017.pdf).

* [Rcpp note](http://yixuan.cos.name/rcpp-note/index.html).

* [Rcpp quick reference guide](http://dirk.eddelbuettel.com/code/rcpp/Rcpp-quickref.pdf).

* [Learncpp](https://www.learncpp.com/).

* [Cplusplus](http://www.cplusplus.com/).

```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. [H. Wickham, Advanced R, 2014](http://adv-r.had.co.nz/Rcpp.html)
